### Desafio 2
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

### Datos
Utilizaremos como dataset la biblia.

In [4]:
import pandas as pd

# Leer el archivo txt línea por línea
with open('bible.txt', 'r', encoding='utf-8') as file:
    lines = file.readlines()

# Crear un DataFrame con las líneas de texto
df = pd.DataFrame(lines, columns=['text'])


### 1 - Preprocesamiento

In [5]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

<ipython-input-5-99a399a8b871>:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sentence_tokens.append(text_to_word_sequence(row[0]))


In [6]:
# Demos un vistazo
sentence_tokens[:2]

[['1',
  '1',
  'in',
  'the',
  'beginning',
  'god',
  'created',
  'the',
  'heaven',
  'and',
  'the',
  'earth'],
 []]

### 2 - Crear los vectores (word2vec)

In [7]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [8]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [9]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [10]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 100182


In [11]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 5546


### 3 - Entrenar embeddings

In [12]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 4777817.0
Loss after epoch 1: 3815242.0
Loss after epoch 2: 3569690.0
Loss after epoch 3: 3522183.0
Loss after epoch 4: 3387094.0
Loss after epoch 5: 3311360.0
Loss after epoch 6: 3281536.0
Loss after epoch 7: 3259660.0
Loss after epoch 8: 3244736.0
Loss after epoch 9: 3217926.0
Loss after epoch 10: 3161648.0
Loss after epoch 11: 3142352.0
Loss after epoch 12: 3139160.0
Loss after epoch 13: 3133480.0
Loss after epoch 14: 3129908.0
Loss after epoch 15: 3132104.0
Loss after epoch 16: 3148856.0
Loss after epoch 17: 3166680.0
Loss after epoch 18: 3213112.0
Loss after epoch 19: 3288852.0


(11676694, 17106420)

### 4 - Ensayar

In [13]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["god"], topn=10)

[('lord', 0.5730342268943787),
 ('redeemer', 0.5453082323074341),
 ('frame', 0.5271272659301758),
 ('saviour', 0.5261989235877991),
 ('chastisement', 0.5240002274513245),
 ('mysteries', 0.5144867300987244),
 ('creator', 0.49918073415756226),
 ('perfected', 0.49882596731185913),
 ('manifold', 0.49732062220573425),
 ('battles', 0.4926450848579407)]

In [32]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["angels"], topn=10)

[('waste', 0.014200050383806229),
 ('wast', 0.0025252443738281727),
 ('deal', -0.006110143382102251),
 ('sore', -0.01044493168592453),
 ('required', -0.010514795780181885),
 ('cedar', -0.012273339554667473),
 ('buy', -0.013524515554308891),
 ('instead', -0.02326493337750435),
 ('elishama', -0.03137492015957832),
 ('wailing', -0.031655602157115936)]

In [15]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["devil"], topn=10)

[('virtue', 0.5468472838401794),
 ('satan', 0.5166077613830566),
 ('depths', 0.5162392854690552),
 ('scorner', 0.5110312104225159),
 ('hypocrite', 0.4994121789932251),
 ('haughtiness', 0.49401673674583435),
 ('dragon', 0.48691684007644653),
 ('worshippeth', 0.47627192735671997),
 ('assyrian', 0.4746253788471222),
 ('thief', 0.4734523594379425)]

In [33]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["jesus"], topn=5)

[('nathanael', 0.5209063291549683),
 ('delilah', 0.4736468195915222),
 ('thessalonians', 0.4716348946094513),
 ('nicodemus', 0.4562121629714966),
 ('sufferings', 0.4494789242744446)]

In [17]:
# Ensayar con una palabra que no está en el vocabulario:
w2v_model.wv.most_similar(negative=["live"])

[('cherubims', 0.05353224650025368),
 ('upper', 0.035112038254737854),
 ('daughter', 0.03446437790989876),
 ('damascus', 0.025203648954629898),
 ('shittim', 0.009352218359708786),
 ('image', 0.0007316727424040437),
 ('linen', -0.009695431217551231),
 ('chamber', -0.01615547388792038),
 ('golden', -0.018067626282572746),
 ('apparel', -0.018889186903834343)]

In [18]:
# el método `get_vector` permite obtener los vectores:
vector_love = w2v_model.wv.get_vector("love")
print(vector_love)

[ 0.09039005  0.05372788 -0.26714545 -0.13950245 -0.22212401 -0.7992531
 -0.11710159  0.2243005   0.18414569 -0.4809237  -0.22478873 -0.28210968
 -0.05582813 -0.44710064 -0.2290846  -0.09007631 -0.27776662  0.40626135
 -0.19397128 -0.26435834 -0.08926313  0.04539828  0.2297129   0.09748125
  0.12479744 -0.08310294 -0.50724727  0.10955284 -0.4955615  -0.19359046
 -0.2495866  -0.4810181   0.04106821 -0.51638544  0.11580996  0.12613517
  0.292876   -0.11123432  0.16893478  0.06195729 -0.19424915  0.03732344
  0.33799672 -0.3119915   0.28347555  0.15507935  0.00507754  0.30813685
 -0.17305163 -0.25167558  0.06347592 -0.14956717 -0.02762952  0.40801656
 -0.22168082  0.04242125 -0.17952333  0.01369147  0.11931366  0.18440315
  0.0552565  -0.35736537  0.5594959  -0.0053529   0.49043342 -0.05448807
  0.29457343  0.14696535 -0.14236319  0.10583593 -0.08996093  0.09630919
  0.24716245  0.13586035  0.2819606  -0.35285553  0.20003062 -0.3073448
  0.09643014  0.38266748 -0.38865313  0.12099951 -0.3

In [19]:
# el método `most_similar` también permite comparar a partir de vectores
w2v_model.wv.most_similar(vector_love)

[('love', 1.0),
 ('direct', 0.4883764684200287),
 ('defraud', 0.4770340025424957),
 ('lovingkindness', 0.47489625215530396),
 ('brotherly', 0.47010743618011475),
 ('speeches', 0.4693288207054138),
 ('instruct', 0.46356064081192017),
 ('grant', 0.4588417410850525),
 ('specially', 0.4569951593875885),
 ('patient', 0.4567239582538605)]

In [20]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["love"], topn=10)

[('direct', 0.4883764684200287),
 ('defraud', 0.4770340025424957),
 ('lovingkindness', 0.47489625215530396),
 ('brotherly', 0.47010743618011475),
 ('speeches', 0.46932876110076904),
 ('instruct', 0.46356064081192017),
 ('grant', 0.4588417410850525),
 ('specially', 0.4569951593875885),
 ('patient', 0.4567239582538605),
 ('confidence', 0.456155389547348)]

Vemos que las palabras más relacionadas coinciden bastante

### 5 - Visualizar agrupación de vectores

In [21]:
from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
import numpy as np

def reduce_dimensions(model, num_dimensions = 2 ):

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [22]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

In [23]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show(renderer="colab") # esto para plotly en colab

In [24]:
# También se pueden guardar los vectores y labels como tsv para graficar en
# http://projector.tensorflow.org/


vectors = np.asarray(w2v_model.wv.vectors)
labels = list(w2v_model.wv.index_to_key)

np.savetxt("vectors.tsv", vectors, delimiter="\t")

with open("labels.tsv", "w") as fp:
    for item in labels:
        fp.write("%s\n" % item)

Vamos a probar una analogía teniendo en cuenta las siguientes palabras: jesus, angeles, discipulos

In [42]:
# Find the 10 most similar words to the analogy 'jesus' + 'angels' - 'disciples'
result = w2v_model.wv.most_similar(positive=['jesus', 'angels'], negative=['disciples'], topn=10)

# Print the results
print("10 words most similar to the analogy 'jesus' + 'angels' - 'disciples':")
for word, similarity in result:
    print(f"{word}: {similarity}")

10 words most similar to the analogy 'jesus' + 'angels' - 'disciples':
pertain: 0.37585151195526123
thessalonians: 0.37351593375205994
trieth: 0.3688833713531494
angel: 0.3578636348247528
chastisement: 0.35320091247558594
secrets: 0.35265064239501953
immortality: 0.35019880533218384
marked: 0.3490939736366272
reproaches: 0.34480902552604675
beforehand: 0.34401026368141174


De las analogías confluimos que existe cierta cercanía semántica en el espacio vectorial entre las palabras resultantes.